In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, 
    GlobalAveragePooling2D, RandomFlip, RandomRotation
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set seeds for reproducibility to ensure consistent results across runs
tf.random.set_seed(42)
np.random.seed(42)

# Paths for dataset
CSV_PATH = "../Dataset/image_labels.csv"  # CSV file containing image paths and labels
IMG_DIR = "../Dataset/interior"           # Directory containing the images
IMG_HEIGHT, IMG_WIDTH = 224, 224          # Image dimensions for resizing
BATCH_SIZE = 80                           # Batch size for training
EPOCHS = 140                              # Maximum number of epochs for training

# Generate a CSV file mapping image paths to their labels
# This function scans the image directory, matches filenames to class labels, and saves the mapping to a CSV
def regenerate_csv(image_dir, output_file):
    # Define the classes we want to classify
    classes = ['bath', 'bed', 'dining room', 'kitchen', 'living room']
    # Variations of class names to handle different filename patterns
    class_variations = {
        'bath': ['bath', 'bathroom'], 'bed': ['bed', 'bedroom'],
        'dining room': ['dining', 'dining_room', 'diningroom', 'din'],
        'kitchen': ['kitchen'], 'living room': ['living', 'living_room', 'livingroom']
    }
    data = []
    # Iterate through all files in the image directory
    for filename in os.listdir(image_dir):
        # Check if the file is an image (jpg, jpeg, or png)
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            matched = False
            # Try to match the filename to a class based on variations
            for cls in classes:
                for variation in class_variations[cls]:
                    if variation.lower() in filename.lower():
                        # If matched, add the image path and label to the data list
                        data.append({'image_path': os.path.join(image_dir, filename), 'label': cls})
                        matched = True
                        break
                if matched:
                    break
    # Convert the data list to a DataFrame and save it as a CSV
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)
    print(f"Regenerated CSV with {len(df)} images")
    return df

# Verify that all image paths in the CSV exist
def verify_data(csv_path, img_dir):
    df = pd.read_csv(csv_path)
    print(f"Total images: {len(df)}, Classes: {df['label'].value_counts()}")
    # Check for missing files
    missing = [path for path in df['image_path'] if not os.path.exists(path)]
    if missing:
        raise ValueError(f"Missing files: {missing}")

# Load and preprocess the dataset for training
def load_and_preprocess_data(csv_path, img_dir):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_path)
    # Encode the string labels (e.g., 'bath') into integers (e.g., 0, 1, ...)
    label_encoder = LabelEncoder()
    df['label_encoded'] = label_encoder.fit_transform(df['label'])
    num_classes = len(label_encoder.classes_)  # Number of unique classes (should be 5)

    # Split the data into training (80%) and validation (20%) sets, ensuring class balance
    train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
    
    # Compute class weights to handle slight imbalances in the dataset
    class_weights = compute_class_weight('balanced', classes=np.unique(df['label_encoded']), y=df['label_encoded'])
    class_weight_dict = dict(enumerate(class_weights))

    # Helper function to normalize images 
    def load_image(image_path, label):
        # Read the image file
        img = tf.io.read_file(image_path)
        # Decode the image as JPEG with 3 color channels (RGB)
        img = tf.image.decode_jpeg(img, channels=3)
        # Convert the pixel values to float32
        img = tf.cast(img, tf.float32)
        # Normalize pixel values to [0, 1]
        img = img / 255.0
        return img, label

    # Create TensorFlow datasets for training and validation
    # Map the image loading function, cache data in memory, shuffle, batch, and prefetch for efficiency
    train_dataset = tf.data.Dataset.from_tensor_slices(
        (train_df['image_path'], tf.keras.utils.to_categorical(train_df['label_encoded'], num_classes))
    ).map(load_image, num_parallel_calls=tf.data.AUTOTUNE).cache().shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    
    val_dataset = tf.data.Dataset.from_tensor_slices(
        (val_df['image_path'], tf.keras.utils.to_categorical(val_df['label_encoded'], num_classes))
    ).map(load_image, num_parallel_calls=tf.data.AUTOTUNE).cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    
    return train_dataset, val_dataset, num_classes, label_encoder, val_df, class_weight_dict

# Build the Model
def build_model(num_classes):
    # Initialize a sequential model (stack of layers)
    model = Sequential([
        # Data Augmentation Layers: These are applied during training to prevent overfitting
        # Randomly flip images horizontally to increase dataset variability
        RandomFlip("horizontal", seed=42),
        # Randomly rotate images by up to 10% (0.1 radians) to make the model robust to rotations
        RandomRotation(0.1, seed=42),

        # First Convolutional Block
        # Conv2D: Apply 32 filters of size 3x3, with 'same' padding to maintain input dimensions
        # Activation: ReLU introduces non-linearity
        # Input Shape: Images are 224x224 with 3 color channels (RGB)
        # L2 Regularization: Penalizes large weights to prevent overfitting
        Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), kernel_regularizer=l2(0.001)),
        # BatchNormalization: Normalizes layer outputs to stabilize and accelerate training
        BatchNormalization(),
        # MaxPooling2D: Downsamples the feature maps by taking the maximum value in 2x2 regions, reducing spatial dimensions
        MaxPooling2D((2, 2)),
        # Dropout: Randomly sets 45% of the units to 0 during training to prevent overfitting
        Dropout(0.45),

        # Second Convolutional Block
        # Increase to 64 filters to capture more complex features
        Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.45),

        # Third Convolutional Block
        # Increase to 128 filters for deeper feature extraction
        Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.45),

        # Fourth Convolutional Block
        # Increase to 256 filters to capture high-level features
        Conv2D(256, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.45),

        # Fifth and Sixth Convolutional Blocks
        # Two consecutive Conv2D layers with 256 filters to deepen the network
        Conv2D(256, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Conv2D(256, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),

        # Global Average Pooling: Reduces spatial dimensions (e.g., 14x14x256) to a 1D vector (256,)
        # This avoids the need for flattening and reduces the number of parameters
        GlobalAveragePooling2D(),

        # Fully Connected Layer
        # Dense layer with 256 units to learn complex patterns from the pooled features
        # L2 regularization to prevent overfitting
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        # High dropout rate (70%) to further combat overfitting
        Dropout(0.7),

        # Output Layer
        # Dense layer with 'num_classes' units (5 in this case) for classification
        # Softmax activation to output probabilities for each class
        Dense(num_classes, activation='softmax')
    ])
    return model

# Plot the training history (accuracy and loss)
def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    for metric in ['accuracy', 'loss']:
        plt.subplot(1, 2, 1 if metric == 'accuracy' else 2)
        plt.plot(history.history[metric], label=f'Training {metric.capitalize()}')
        plt.plot(history.history[f'val_{metric}'], label=f'Validation {metric.capitalize()}')
        plt.title(f'Training and Validation {metric.capitalize()}')
        plt.legend()
    plt.savefig('training_history.png')
    plt.close()

# Plot the confusion matrix to visualize model performance
def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig('confusion_matrix.png')
    plt.close()

# Main function to execute the pipeline
def main():
    # Generate the CSV file with image paths and labels
    df = regenerate_csv(IMG_DIR, CSV_PATH)
    if df is None:
        return
    
    # Verify that all image paths exist
    verify_data(CSV_PATH, IMG_DIR)
    
    # Load and preprocess the dataset
    train_dataset, val_dataset, num_classes, label_encoder, val_df, class_weight_dict = load_and_preprocess_data(CSV_PATH, IMG_DIR)
    
    # Build the model
    model = build_model(num_classes)
    
    # Compile the model
    # Optimizer: Adam with a learning rate of 0.001
    # Loss: Categorical crossentropy for multi-class classification
    # Metrics: Track accuracy during training
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define callbacks to improve training
    callbacks = [
        # EarlyStopping: Stop training if validation loss doesn't improve for 30 epochs
        # Restore the best weights to avoid overfitting
        EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True),
        # ReduceLROnPlateau: Reduce learning rate by a factor of 0.6 if validation loss plateaus for 6 epochs
        ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=6, min_lr=1e-6),
        # CSVLogger: Log training metrics (e.g., loss, accuracy) to a CSV file
        CSVLogger('training_log.csv', append=True)
    ]
    
    # Train the model
    # Use the training dataset, validate on the validation dataset
    # Apply class weights to handle imbalance
    history = model.fit(
        train_dataset, epochs=EPOCHS, validation_data=val_dataset,
        callbacks=callbacks, class_weight=class_weight_dict, verbose=1
    )
    
    # Evaluate the model on the validation set
    val_loss, val_accuracy = model.evaluate(val_dataset)
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")
    
    # Save the trained model
    model.save('model.keras')
    
    # Save the label encoder classes for later use
    np.save('label_encoder_classes.npy', label_encoder.classes_)
    
    # Plot training and validation accuracy/loss
    plot_training_history(history)
    
    # Get sample predictions to inspect model performance
    val_images, val_labels = next(iter(val_dataset))
    predictions = model.predict(val_images)
    predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))
    true_labels = label_encoder.inverse_transform(np.argmax(val_labels, axis=1))
    print("Sample Predictions:", *[(t, p) for t, p in zip(true_labels[:10], predicted_labels[:10])], sep='\n')
    
    # Generate a classification report and confusion matrix
    val_predictions = model.predict(val_dataset)
    val_pred_labels = np.argmax(val_predictions, axis=1)
    val_true_labels = np.argmax(np.concatenate([y for _, y in val_dataset]), axis=1)
    report = classification_report(val_true_labels, val_pred_labels, target_names=label_encoder.classes_)
    print("\nClassification Report:\n", report)
    
    # Save the classification report to a file
    with open('classification_report.txt', 'w') as f:
        f.write(report)
    
    # Plot and save the confusion matrix
    plot_confusion_matrix(val_true_labels, val_pred_labels, label_encoder.classes_)

if __name__ == "__main__":
    main()

Num GPUs Available:  1
Regenerated CSV with 12335 images
Total images: 12335, Classes: label
living room    2621
dining room    2605
bed            2445
bath           2430
kitchen        2234
Name: count, dtype: int64
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip (RandomFlip)    (None, 224, 224, 3)       0         
                                                                 
 random_rotation (RandomRota  (None, 224, 224, 3)      0         
 tion)                                                           
                                                                 
 conv2d_25 (Conv2D)          (None, 224, 224, 32)      896       
                                                                 
 batch_normalization_24 (Bat  (None, 224, 224, 32)     128       
 chNormalization)                                                
                                   

2025-05-21 03:08:07,413 - WARNING - Using a while_loop for converting RngReadAndSkip cause there is no registered converter for this op.


2025-05-21 03:08:07,475 - WARNING - Using a while_loop for converting Bitcast cause there is no registered converter for this op.


2025-05-21 03:08:07,544 - WARNING - Using a while_loop for converting Bitcast cause there is no registered converter for this op.


2025-05-21 03:08:07,601 - WARNING - Using a while_loop for converting StatelessRandomUniformV2 cause there is no registered converter for this op.


...

2025-05-21 03:08:12,531 - WARNING - Using a while_loop for converting Bitcast cause there is no registered converter for this op.


2025-05-21 03:08:12,609 - WARNING - Using a while_loop for converting Bitcast cause there is no registered converter for this op.


2025-05-21 03:08:12,673 - WARNING - Using a while_loop for converting StatelessRandomUniformV2 cause there is no registered converter for this op.


2025-05-21 03:08:13,240 - WARNING - Using a while_loop for converting ImageProjectiveTransformV3 cause there is no registered converter for this op.


124/124 [==============================] - 158s 953ms/step - loss: 2.6273 - accuracy: 0.3166 - val_loss: 6.8446 - val_accuracy: 0.1970 - lr: 0.0010
Epoch 2/140
124/124 [==============================] - 123s 978ms/step - loss: 2.2688 - accuracy: 0.3807 - val_loss: 10.0422 - val_accuracy: 0.1970 - lr: 0.0010
Epoch 3/140
124/124 [==============================] - 102s 819ms/step - loss: 1.9826 - accuracy: 0.4379 - val_loss: 4.6576 - val_accuracy: 0.1986 - lr: 0.0010
Epoch 4/140
124/124 [==============================] - 103s 833ms/step - loss: 1.7753 - accuracy: 0.4874 - val_loss: 4.7322 - val_accuracy: 0.2011 - lr: 0.0010
Epoch 5/140
124/124 [==============================] - 102s 820ms/step - loss: 1.6195 - accuracy: 0.5281 - val_loss: 2.8307 - val_accuracy: 0.3129 - lr: 0.0010
Epoch 6/140
124/124 [==============================] - 102s 821ms/step - loss: 1.5216 - accuracy: 0.5565 - val_loss: 1.8651 - val_accuracy: 0.4349 - lr: 0.0010
Epoch 7/140
124/124 [==============================